In [12]:
# The path can also be read from a config file, etc.
import dotenv
OPENSLIDE_PATH = dotenv.get_key(dotenv.find_dotenv(), "OPENSLIDE_PATH")
print(OPENSLIDE_PATH)

D:\Benya\Downloads\openslide-win64-20231011\openslide-win64-20231011\bin


In [13]:
import sys
import torch
sys.path.append('../..')
from src.models import ResNet34
from torchvision.transforms import ToTensor
from torch import nn,Tensor
from tqdm import tqdm
from itertools import product
import os
#if on linux just import openslide directly
if hasattr(os, 'add_dll_directory'):
    # Windows
    with os.add_dll_directory(OPENSLIDE_PATH):
        import openslide
        from openslide import open_slide
        from openslide.deepzoom import DeepZoomGenerator
else:
    import openslide
    from openslide import open_slide
    from openslide.deepzoom import DeepZoomGenerator

In [14]:
WSI_PATH = "D:\\AIDS\\S2\\Project\\Dataset\\WSI\\BRACS_1003694.svs"

In [15]:
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"device = {device}")

device = cuda


In [16]:
patch = 224

model = ResNet34(n_classes=3).to(device)

In [17]:
def get_vector(model : nn.Module, patch : Tensor):

    y = model.conv1(patch)
    y = model.bn1(y)
    y = model.relu(y)
    y = model.maxpool(y)
    y = model.layer1(y)
    y = model.layer2(y)
    y = model.layer3(y)
    y = model.layer4(y)
    y = model.avgpool(y)
    y = torch.flatten(y)

    return y

In [18]:
slide = open_slide(WSI_PATH)
print(slide.level_dimensions)
tiles = DeepZoomGenerator(slide,tile_size=patch,overlap=0,limit_bounds=False)

((125663, 41933), (31415, 10483), (7853, 2620), (3926, 1310))


In [19]:
W, H = tiles.level_tiles[tiles.level_count - 1]

print(W,H)

561 188


In [20]:
to_tensor = ToTensor()

model.eval()

batch_size = 256

vecs = []

batch = []

i = 0

In [23]:
with torch.inference_mode():

    for w,h in tqdm(list(product(range(W - 1),range(H - 1)))):

        if len(batch) < batch_size:
            tile = tiles.get_tile(level=tiles.level_count-1,address=(w,h))
            tile = to_tensor(tile)
            batch.append(tile)
        else:
            print(type(batch))
            batch = torch.stack(batch).to(device)
            vec = get_vector(model.resnet ,batch)
            vecs.append(vec)
            batch = []

  0%|          | 0/104720 [00:00<?, ?it/s]

<class 'torch.Tensor'>


TypeError: stack(): argument 'tensors' (position 1) must be tuple of Tensors, not Tensor